<a href="https://colab.research.google.com/github/royam0820/crewai/blob/main/L6_collaboration_financial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
!pip install crewai==0.28.8
!pip install crewai_tools==0.1.6
!pip install langchain_community==0.0.29 # this is formatted as code
!pip install utils

  Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.29
    Uninstalling langchain-community-0.0.29:
      Successfully uninstalled langchain-community-0.0.29
  Using cached langchain_community-0.0.29-py3-none-any.whl (1.8 MB)
  Attempting uninstall: langchain_community
    Found existing installation: langchain-community 0.0.38
    Uninstalling langchain-community-0.0.38:
      Successfully uninstalled langchain-community-0.0.38
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-community<0.1,>=0.0.38, but you have langchain-community 0.0.29 which is incompatible.


In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [4]:
from crewai import Agent, Task, Crew

In [6]:
import os
from google.colab import userdata

In [7]:
openai_api_key = userdata.get("OPENAI_API_KEY")
serper_api_key = userdata.get("SERPER_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

In [ ]:
# import os
# from utils import get_openai_api_key, get_serper_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [8]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [9]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [11]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [12]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [13]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [14]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [15]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [16]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [17]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [18]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should delegate tasks to the appropriate team members and ask the necessary questions to gather insights and alerts about significant market opportunities or threats.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
Thought: To continuously monitor and analyze market data for AAPL (Apple Inc.), I need to acce

- Display the final result as Markdown.

In [20]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Proposed Trading Strategies and Execution Plans for AAPL:

After conducting a thorough risk analysis of the proposed trading strategies and execution plans for AAPL, several key risks have been identified along with corresponding mitigation strategies:

1. Market Volatility Risk:
   - **Risk**: The historical price data for AAPL shows significant price fluctuations, indicating market volatility that could impact trading strategies.
   - **Mitigation Strategy**: Implement a diversified trading strategy that includes setting stop-loss orders to manage potential losses. Real-time monitoring of market conditions and adjusting the trading strategy accordingly can help mitigate risks associated with market volatility.

2. Overbought Conditions Risk:
   - **Risk**: Technical indicators such as RSI and CCI suggest that AAPL may be overbought, increasing the risk of a price correction or reversal.
   - **Mitigation Strategy**: Stay updated with AAPL's financial health, market news, and technological innovations to make informed trading decisions. Consider adjusting trading positions based on the user's risk tolerance to mitigate the risks associated with overbought conditions.

3. MACD Sell Signals Risk:
   - **Risk**: The MACD indicator is showing potential sell signals, indicating a bearish trend that could impact trading outcomes.
   - **Mitigation Strategy**: Monitor the MACD closely for any significant changes and consider adjusting the trading strategy accordingly. Implement tight stop-loss orders to manage potential losses in case of a market downturn.

4. Execution Timing Risk:
   - **Risk**: Initiate buy positions based on specific entry points may result in missed opportunities or losses if the execution timing is not optimal.
   - **Mitigation Strategy**: Continuously monitor AAPL's price action throughout the trading day and adapt the execution timing based on real-time market conditions. Consider setting take-profit levels and stop-loss orders to manage risks associated with execution timing.

In conclusion, to effectively manage the risks associated with the proposed trading strategies and execution plans for AAPL, it is essential to implement a diversified trading strategy, stay informed about market conditions, adjust trading positions based on risk tolerance, and monitor technical indicators closely. By following these mitigation strategies, the potential risks can be minimized, and the trading outcomes can be optimized for successful trading of AAPL stock.

In [ ]:
import os
from google.colab import userdata

In [ ]:
openai_api_key = userdata.get("OPENAI_API_KEY")
serper_api_key = userdata.get("SERPER_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

In [ ]:
import os
from google.colab import userdata

In [ ]:
openai_api_key = userdata.get("OPENAI_API_KEY")
serper_api_key = userdata.get("SERPER_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'